In [1]:
import numpy as np
import pandas as pd

# Модель для прогнозирования, например ARIMA
from sktime.forecasting.arima import ARIMA
# Визуализация временных рядов
from sktime.utils.plotting import plot_series
# Модули для кросс-валидации
from sktime.split import temporal_train_test_split, ExpandingWindowSplitter, SlidingWindowSplitter, SingleWindowSplitter
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError # Метрики MSE, MAE, MAPE
# Поиск оптимальных гиперпараметров по сетке
from sktime.forecasting.model_selection import ForecastingGridSearchCV

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

In [2]:
y = np.log(web.DataReader(name='GDP', data_source='fred', start='1995-01-01'))
y.index = y.index.to_period(freq='M') # замена индекса на периодический. Иначе кросс-валидация не будет работать
# длина ряда
len(y)

120

## Проведем Grid Search

In [5]:
# Зададим метод прогнозирования
forecaster = ARIMA()

# разбиваем параметры кросс-валидации
cv_strategy = ExpandingWindowSplitter(fh=np.arange(1, 6), initial_window=80, step_length=1)

# Задаём сетку для значений параметров модели в виде словаря
# будем менять параметры order и trend
param_grid = {'order':[(1,0,1), (1,1,0), (1,1,1), (1,2,0), (0,1,1), (0,2,0), (1,1,1)],
               'trend': ['ct', 'c', 'c', 'n', 'c', 'n', 'c']}

# инициализируем метрики
metric = MeanSquaredError(square_root=False)

# Grid search - метод поиска
gscv = ForecastingGridSearchCV(forecaster=forecaster, param_grid=param_grid, cv=cv_strategy, scoring=metric)

gscv.fit(y)

ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=array([1, 2, 3, 4, 5]),
                                                   initial_window=80),
                        forecaster=ARIMA(),
                        param_grid={'order': [(1, 0, 1), (1, 1, 0), (1, 1, 1),
                                              (1, 2, 0), (0, 1, 1), (0, 2, 0),
                                              (1, 1, 1)],
                                    'trend': ['ct', 'c', 'c', 'n', 'c', 'n',
                                              'c']},
                        scoring=MeanSquaredError())

In [6]:
# Параметры оптимальной модели
gscv.get_fitted_params()['best_forecaster']

ARIMA(order=(0, 1, 1), trend='c')

In [8]:
# Зададим метод прогнозирования
forecaster = ARIMA()

# разбиваем параметры кросс-валидации
cv_strategy = ExpandingWindowSplitter(fh=np.arange(1, 6), initial_window=80, step_length=1)

# Задаём сетку для значений параметров модели в виде словаря
# будем менять параметры order и trend
param_grid = {'order':[(1,0,1), (1,1,0), (1,1,1), (1,2,0), (0,1,1), (0,2,0), (1,1,1)],
               'trend': ['ct', 'c', 'c', 'n', 'c', 'n', 'c']}

# инициализируем метрики
metric = MeanAbsoluteError()

# Grid search - метод поиска
gscv = ForecastingGridSearchCV(forecaster=forecaster, param_grid=param_grid, cv=cv_strategy, scoring=metric)

gscv.fit(y)

ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=array([1, 2, 3, 4, 5]),
                                                   initial_window=80),
                        forecaster=ARIMA(),
                        param_grid={'order': [(1, 0, 1), (1, 1, 0), (1, 1, 1),
                                              (1, 2, 0), (0, 1, 1), (0, 2, 0),
                                              (1, 1, 1)],
                                    'trend': ['ct', 'c', 'c', 'n', 'c', 'n',
                                              'c']},
                        scoring=MeanAbsoluteError())

In [9]:
# Параметры оптимальной модели
gscv.get_fitted_params()['best_forecaster']

ARIMA(order=(0, 1, 1), trend='c')

### Построим оптимальную модель ARIMA для USM2

In [10]:
y = np.log(web.DataReader(name='MYAGM2USM052S', data_source='fred', start='1995-01-01'))
y.index = y.index.to_period(freq='M') # замена индекса на периодический. Иначе кросс-валидация не будет работать
# длина ряда
len(y)

267

In [11]:
# Зададим метод прогнозирования
forecaster = ARIMA()

# разбиваем параметры кросс-валидации
cv_strategy = ExpandingWindowSplitter(fh=np.arange(1, 6), initial_window=100, step_length=1)

# Задаём сетку для значений параметров модели в виде словаря
# будем менять параметры order и trend
param_grid = {'order':[(2,0,2), (2,1,0), (2,1,1), (1,2,0)],
               'trend': ['ct', 'c', 'n', 'n']}

# инициализируем метрики
metric = MeanAbsoluteError()

# Grid search - метод поиска
gscv = ForecastingGridSearchCV(forecaster=forecaster, param_grid=param_grid, cv=cv_strategy, scoring=metric)

gscv.fit(y)

ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=array([1, 2, 3, 4, 5]),
                                                   initial_window=100),
                        forecaster=ARIMA(),
                        param_grid={'order': [(2, 0, 2), (2, 1, 0), (2, 1, 1),
                                              (1, 2, 0)],
                                    'trend': ['ct', 'c', 'n', 'n']},
                        scoring=MeanAbsoluteError())

In [13]:
gscv.get_fitted_params()['best_forecaster']

ARIMA(order=(2, 0, 2), trend='ct')

In [14]:
gscv.get_params

<bound method BaseObject.get_params of ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=array([1, 2, 3, 4, 5]),
                                                   initial_window=100),
                        forecaster=ARIMA(),
                        param_grid={'order': [(2, 0, 2), (2, 1, 0), (2, 1, 1),
                                              (1, 2, 0)],
                                    'trend': ['ct', 'c', 'n', 'n']},
                        scoring=MeanAbsoluteError())>